In [1]:
import pandas as pd
pd.options.display.max_columns = None
import re
import numpy as np

Seguiremos trabajando con mismos csv que en el pair anterior, el de los empleados que unimos en el pair de unión de datasets y el de los tickets de todos los años. Para ello, lo primero que tendremos que hacer es cargar estos dos csv.

1. Cread una columna nueva y una función que nos de el mes en el que se solicitó el servicio. Tened en cuenta que todas las filas tienen la misma estructura. A la hora de escribir la función debemos tener en cuenta que puede que no haya la información de mes en la columna de la fecha. En ese caso devolved un nulo (NaN).

📌 Nota La fecha tendrá que ir formato de string como "Enero", "Febrero", etc.

💡 Pista 💡Podréis usar regex, pero no es estrictamente necesario.

In [2]:
df_pe = pd.read_csv("datos/mergeado.csv") #puestos-empleados
df_pe.head(1)

,Unnamed: 0,ID Puesto,Puesto,Departamento,Tipo Puesto,Turno,ID Empleado,Apellido,Nombre,Email
0,0,10-23200,Ingeniero de Materiales,MATERIALES,Operativo,NaN,725,Villa,Eliana,eliana .villa@tractchun.com


In [3]:
df_copia_pe= df_pe.copy() #copia puestos-empleados
df_copia_pe.head(1)

,Unnamed: 0,ID Puesto,Puesto,Departamento,Tipo Puesto,Turno,ID Empleado,Apellido,Nombre,Email
0,0,10-23200,Ingeniero de Materiales,MATERIALES,Operativo,NaN,725,Villa,Eliana,eliana .villa@tractchun.com


In [4]:
df_tickets = pd.read_csv("datos/concatenado.csv") #tiquets
df_tickets.head(1)

,Unnamed: 0,ID Ticket,Fecha,ID Empleado,ID Agente,Categoría,Tipo,Severidad,Prioridad,Días Resolución,Satisfacción,año
0,0,GDDENR-5042564453,2016-07-13,1735.0,4.0,Acceso&%Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016


In [5]:
df_copia_ticket= df_tickets.copy() #copia tiquets
df_copia_ticket.head(1)

,Unnamed: 0,ID Ticket,Fecha,ID Empleado,ID Agente,Categoría,Tipo,Severidad,Prioridad,Días Resolución,Satisfacción,año
0,0,GDDENR-5042564453,2016-07-13,1735.0,4.0,Acceso&%Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016


In [6]:
df_copia_ticket['Fecha'].isnull().sum() #comprobamos que hay 0 nulos

0

In [7]:
df_copia_ticket['Fecha']= pd.to_datetime(df_copia_ticket['Fecha']) #pasamos a fecha el formato str

In [8]:
def sacar_mes(x): #función para extraer el mes 
        try:
                return x.strftime("%B")
        except:
            return np.nan
                

In [9]:
df_copia_ticket["mes"] = df_copia_ticket["Fecha"].apply(sacar_mes) #apply para añadir la columna mes con el resultado de la función ejecutada
df_copia_ticket.head(1)

,Unnamed: 0,ID Ticket,Fecha,ID Empleado,ID Agente,Categoría,Tipo,Severidad,Prioridad,Días Resolución,Satisfacción,año,mes
0,0,GDDENR-5042564453,2016-07-13,1735.0,4.0,Acceso&%Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016,July


2. En la tabla de empleados, tenemos los nombres y los apellidos de los empleados, pero en columnas separadas. En los ejercicios de pair de la lección del groupby vimos que si quisieramos agrupar por los nombres, esto se nos puede complicar. Ahora nos piden que creemos una columna donde tengamos el nombre y el apellido juntos.

📌 NOTA La columna nueva debe tener el nombre y apellido con la primera letra en mayúsculas, por ejemplo "Jesús Villa".

📌 NOTA Una vez hallamos creado la columna nueva, eliminad las columnas de nombre y apellido originales.

In [10]:
df_copia_pe.head(1)

,Unnamed: 0,ID Puesto,Puesto,Departamento,Tipo Puesto,Turno,ID Empleado,Apellido,Nombre,Email
0,0,10-23200,Ingeniero de Materiales,MATERIALES,Operativo,NaN,725,Villa,Eliana,eliana .villa@tractchun.com


In [11]:
def sacar_nombre(nombre, apellido): #función para sumar las columnas Nombre y Apellidos, diciéndole que las primeras letras vayan en mayúsculas

    return nombre.title() + ' ' + apellido.title()


In [12]:
df_copia_pe['Nombre y Apellidos']= df_copia_pe.apply(lambda dataframe: sacar_nombre(dataframe['Nombre'], dataframe['Apellido']), axis=1 ) 
#añadimos la columna, que surge a partir del proceso apply


In [14]:
df_copia_pe.sample(1)

,Unnamed: 0,ID Puesto,Puesto,Departamento,Tipo Puesto,Turno,ID Empleado,Apellido,Nombre,Email,Nombre y Apellidos
19,19,10-32100,Supervisor de Almacén,MATERIALES,Mandos Medios,NaN,891,GONZALES,DARIO,dario.gonzales@tractchun.com,Dario Gonzales


In [15]:
df_copia_pe.drop(['Apellido', 'Nombre'], axis=1, inplace=True) #eliminamos las columnas

In [16]:
df_copia_pe.head(1)

,Unnamed: 0,ID Puesto,Puesto,Departamento,Tipo Puesto,Turno,ID Empleado,Email,Nombre y Apellidos
0,0,10-23200,Ingeniero de Materiales,MATERIALES,Operativo,NaN,725,eliana .villa@tractchun.com,Eliana Villa


3. En el pair programming desde a dirección nos pedían que agruparamos los datos en base a la columna de "categoría", puede que en aquel momento nos dieramos cuenta que había varías categorías que eran igual (en concreto "Acceso Login") pero que en algunos casos tenía unos símbolos raros (&, %, -). Es el momento de limpiar esta columna. El objetivo, reemplazar todos estos símbolos raros por "/", para conseguir homogeneizar los valores de esta columna. Para ellos deberemos crear una función.

In [17]:
df_copia_ticket.head(10)

,Unnamed: 0,ID Ticket,Fecha,ID Empleado,ID Agente,Categoría,Tipo,Severidad,Prioridad,Días Resolución,Satisfacción,año,mes
0,0,GDDENR-5042564453,2016-07-13,1735.0,4.0,Acceso&%Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016,July
1,1,GDDENR-8042508060,2016-05-18,1566.0,10.0,Acceso&%Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016,May
2,2,GDDESR-1342539995,2016-06-18,569.0,29.0,Sistema,Problema,0 - Sin Clasificar,0 - Sin Asignar,3.0,5.0,2016,June
3,3,GDDTSR-5942488006,2016-04-28,320.0,40.0,Sistema,Solicitud,0 - Sin Clasificar,0 - Sin Asignar,9.0,5.0,2016,April
4,4,GDLEER-0042524120,2016-06-03,1842.0,31.0,Software,Problema,2 - Normal,0 - Sin Asignar,0.0,5.0,2016,June
5,5,GDLEER-0142608095,2016-08-26,59.0,20.0,Software,Problema,2 - Normal,0 - Sin Asignar,1.0,1.0,2016,August
6,6,GDLEER-0242564650,2016-07-13,1175.0,36.0,Software,Problema,2 - Normal,0 - Sin Asignar,2.0,1.0,2016,July
7,7,GDLEER-0542574815,2016-07-23,561.0,18.0,Software,Problema,2 - Normal,0 - Sin Asignar,5.0,5.0,2016,July
8,8,GDLEER-0842457219,2016-03-28,71.0,12.0,Software,Problema,2 - Normal,0 - Sin Asignar,8.0,5.0,2016,March
9,9,GDLEER-1242542213,2016-06-21,1831.0,42.0,Software,Problema,2 - Normal,0 - Sin Asignar,2.0,5.0,2016,June


In [18]:
df_copia_ticket['Categoría'].unique() #comprobamos los elementos que hay dentro de la columna categoría

array(['Acceso&%Login', 'Sistema', 'Software', 'Hardware',
       'Acceso%&Login', 'Acceso-&Login', 'Acceso99Login', 'Acceso/Login'],
      dtype=object)

In [19]:
patron_antiguo = 'Acces.*' #patrón que vemos dentro de la columna
patron_nuevo = 'Acceso/Login' #patrón que queremos que tenga en lel futuro

In [20]:
def simbolos(antiguo, nuevo, columna): #función con tres parámetros 
    try:
        simbolo= re.sub(antiguo, nuevo, columna) #método .sub de regex para reflejar, sustituir, y lugar donde sustituir
        return simbolo
    except:
        return np.nan
    

In [21]:
df_copia_ticket['Categoría']=df_copia_ticket.apply(lambda dataframe: simbolos(patron_antiguo , patron_nuevo, dataframe['Categoría']), axis=1 ) #apply que aplica la función

In [23]:
df_copia_ticket.sample(5) #comprobamos los cambios

,Unnamed: 0,ID Ticket,Fecha,ID Empleado,ID Agente,Categoría,Tipo,Severidad,Prioridad,Días Resolución,Satisfacción,año,mes
31480,31480,GHLTST-8743201637,2018-04-11,513.0,36.0,Sistema,Solicitud,2 - Normal,3 - Alta,7.0,5.0,2018,April
21844,21844,SDLENR-0042771824,2017-02-05,622.0,48.0,Acceso/Login,Problema,2 - Normal,0 - Sin Asignar,0.0,5.0,2017,February
7254,7254,KWLTER-0842443473,2016-03-14,1647.0,24.0,Software,Solicitud,2 - Normal,1 - Baja,8.0,5.0,2016,March
86481,86481,SDLTSR-4544150369,2020-11-15,1046.0,33.0,Sistema,Solicitud,2 - Normal,0 - Sin Asignar,5.0,4.0,2020,November
66319,66319,THLTSR-3543498327,2019-02-02,1012.0,3.0,Sistema,Solicitud,2 - Normal,3 - Alta,3.0,4.0,2019,February


4. Una vez que hayáis terminado eliminad las columnas originales conservando las nuevas.

Proceso de limpieza para el df tiquets

In [24]:
df_copia_ticket.drop(['Unnamed: 0', 'Fecha'], axis= 1, inplace=True) #eliminamos las columnas que no queremos

In [26]:
df_copia_ticket.rename(columns={'ID Ticket' : 'id_ticket', 'ID Empleado':'id_empleado', 'ID Agente':'id_agente', 'Categoría': 'categoría', 'Tipo':'tipo', 'Severidad': 'severidad', 'Prioridad':'prioridad', 'Días Resolución':'dias_resolucion', 'Satisfacción':'satisfaccion' }, inplace=True)
#homogeneizamos el dataframe

In [27]:
df_copia_ticket

,id_ticket,id_empleado,id_agente,categoría,tipo,severidad,prioridad,dias_resolucion,satisfaccion,año,mes
0,GDDENR-5042564453,1735.0,4.0,Acceso/Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016,July
1,GDDENR-8042508060,1566.0,10.0,Acceso/Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016,May
2,GDDESR-1342539995,569.0,29.0,Sistema,Problema,0 - Sin Clasificar,0 - Sin Asignar,3.0,5.0,2016,June
3,GDDTSR-5942488006,320.0,40.0,Sistema,Solicitud,0 - Sin Clasificar,0 - Sin Asignar,9.0,5.0,2016,April
4,GDLEER-0042524120,1842.0,31.0,Software,Problema,2 - Normal,0 - Sin Asignar,0.0,5.0,2016,June
...,...,...,...,...,...,...,...,...,...,...,...
97381,TWRTSR-8543883120,1142.0,1.0,Sistema,Solicitud,3 - Mayor,1 - Baja,11.0,5.0,2020,February
97382,TWRTSR-8744097039,223.0,40.0,Sistema,Solicitud,1 - Menor,1 - Baja,7.0,4.0,2020,September
97383,TWRTSR-9643846768,256.0,7.0,Sistema,Solicitud,3 - Mayor,1 - Baja,13.0,5.0,2020,January
97384,TWRTSR-9944138906,1060.0,9.0,Sistema,Solicitud,1 - Menor,1 - Baja,9.0,5.0,2020,November


Proceso de limpieza para el df puestos-empleados

In [31]:
df_copia_pe.head(1)

,Unnamed: 0,ID Puesto,Puesto,Departamento,Tipo Puesto,Turno,ID Empleado,Email,Nombre y Apellidos
0,0,10-23200,Ingeniero de Materiales,MATERIALES,Operativo,NaN,725,eliana .villa@tractchun.com,Eliana Villa


In [33]:
df_copia_pe.drop(['Unnamed: 0'], axis= 1, inplace=True) #eliminamos columna innecesaria

In [34]:
df_copia_pe.rename(columns={'ID Puesto': 'id_puesto', 'Puesto':'puesto', 'Departamento': 'departamento', 'Tipo Puesto': 'tipo_puesto', 'Turno':'turno', 'ID Empleado': 'id_empleado', 'Email':'email', 'Nombre y Apellidos':'nombre_completo'}, inplace=True)
#homogeneizamos df puesto-empleado

In [36]:
df_copia_pe.sample() #comprobamos

,id_puesto,puesto,departamento,tipo_puesto,turno,id_empleado,email,nombre_completo
617,20-31100,Analista de Sistemas,"SISTEMAS, TRAFICO Y ADUANAS",Administrativo,NaN,1488,jhon .gonzález@tractchun.com,Jhon González


5. Guardad el dataframe en un csv que usaremos más adelante

In [37]:
df_copia_pe.to_csv("datos/puestos-empleados.csv")

In [38]:
df_copia_ticket.to_csv("datos/tiquets.csv")